In [ ]:
from ipyparallel import Client
client = Client(profile='python3')

In [ ]:
from hpc05 import Client
client = Client(profile='pbs', timeout=10)

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
len(dview)

In [ ]:
# Standard library imports
import sys
sys.path.append('/home/basnijholt/Dropbox/Work/nanowire_current/')
sys.path.append('/home/bnijholt/Dropbox/Work/nanowire_current/')
from itertools import product
import types

# Related third party imports
import holoviews as hv
from ipyparallel import require
import kwant
import numpy as np
import pandas as pd
import sympy
import sympy.interactive

# Local imports
import funcs
from funcs import k_B
from funcs import constants

# next two lines because of https://github.com/ipython/ipyparallel/issues/205
%px import sys; sys.path.append('/home/bnijholt/Dropbox/Work/nanowire_current/')
%px import funcs, types, kwant

%matplotlib inline
hv.notebook_extension()
sympy.interactive.init_printing('mathjax')


# 3D test system

In [ ]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz_
onsite_lead = lambda site, p: -p.mu * sz_ + p.delta * sx_
hops = lambda site1, site2, p: -p.t * sz_

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3))
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    range_Z = range(Z)
    syst[(lat(x, y, z) for x in range_X for y in range_Y for z in range_Z)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range_Y for z in range_Z)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

delta = 0.01
T = 0.5 * delta / k_B
syst, hopping = make_test_system(X=3, Y=3, Z=3)
p = types.SimpleNamespace(mu=0., t=1., delta=1)

phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([funcs.current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=None, max_frequencies=200)
              for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# 3D system with Discretizer Hamiltonian, squared system

In [ ]:
def make_test_system(X, Y, Z, a=10):
    tb_normal, tb_sc = funcs.discretized_hamiltonian(a)[:-1]
    lat = tb_normal.lattice
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = tb_normal.onsite

    for hop, val in tb_normal.hoppings.items():
        syst[hop] = val

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = tb_sc.onsite

    for hop, val in tb_sc.hoppings.items():
        lead[hop] = val

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

T = 60e-3

syst, hopping = make_test_system(X=2, Y=3, Z=3)
p = funcs.make_params(Delta=0.25, alpha=20, mu=15)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([funcs.current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=None, max_frequencies=200)
              for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# Simpler way (but slightly slower)

In [ ]:
from funcs import matsubara_frequency, k_B
def null_G(syst, p, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return gf.data[::2, ::2]

def current_from_G_0(T, G_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

matsfreqs = 200
G_0s = [null_G(syst, p, T, n) for n in range(matsfreqs)]
H12 = hopping(syst, args=[p])
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(T, G_0s, H12, phase) for phase in phases]
I = np.array(I)
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# Test 2D system

In [ ]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz_
onsite_lead = lambda site, p: -p.mu * sz_ + p.delta * sx_
hops = lambda site1, site2, p: -p.t * sz_

def make_test_system_2D(X=3, Y=2):
    lat = kwant.lattice.square()
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    syst[(lat(x, y) for x in range_X for y in range_Y)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y) for x, y in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y) for x, y in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0)))
    lead[(lat(0, y) for y in range_Y)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


In [ ]:
syst, _ = make_test_system_2D()
kwant.plot(syst)

# 1D


In [ ]:
syst_pars = dict(a=10, L=50, L_sc=1)
ham_pars = dict(B_y=0, B_z=0, Delta=0.25, g=50, mu_B=constants.mu_B, t=constants.t, V=lambda x: 0)
T = 1
alphas = np.linspace(0, 100, 200)
B_xs = np.linspace(0, 1.5, 200)
mus = [0.1, 0.3 ,1, 3, 10, 30]
vals = list(product(alphas, B_xs, mus))

@require('kwant', 'types', 'funcs')
def fun(x, syst_pars=syst_pars, ham_pars=ham_pars, T=T):
    syst, hopping = funcs.make_1d_wire(**syst_pars)
    p = types.SimpleNamespace(**ham_pars, alpha=x[0], B_x=x[1], mu=x[2])
    return funcs.I_c(syst, hopping, p, T=T)

current_phase = lview.map_async(fun, vals)

current_phase.wait_interactive()

current_phase = current_phase.result()

df = pd.concat((pd.DataFrame(vals, columns=['alpha', 'B_x', 'mu']), pd.DataFrame(current_phase)), axis=1)
df = df.assign(**syst_pars, **ham_pars, T=T).assign(**constants.__dict__)
df = df.assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('1d_alpha_vs_B_x.hdf', 'all_data', mode='w')

In [ ]:
def Image(group, x, y, z):
    bounds = (df[x].min(), df[y].min(), df[x].max(), df[y].max())
    table = group.pivot_table(index=[x], columns=[y], values=[z])
    return hv.Image(np.rot90(table), kdims=[x, y], vdims=[z], bounds=bounds)

In [ ]:
%%opts Image (cmap='viridis') Layout {+axiswise}
ims = [(mu, Image(group, 'B_x', 'alpha', 'current_c')) for mu, group in df.groupby('mu')]
hm = hv.HoloMap(ims, kdims=['mu'])
hv.Layout([im.relabel('mu = {} meV'.format(mu)) for mu, im in ims]).cols(3)

In [ ]:
syst, hopping = funcs.make_1d_wire(a=10, L=30, L_sc=40)
p = funcs.make_params(Delta=0.25, alpha=25, mu=3)
p.mu_sc = p.mu_sm = p.mu
I = np.array([funcs.I_c(syst, hopping, p, T=1)['current_c'] for p.B_x in B_xs])
hv.Curve((B_xs, I), kdims=['B_x'], vdims=['$I_c$'])

In [ ]:
p.mu_sc = p.mu_sm = p.mu
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
T = 1
I = np.array([funcs.current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=1e-2, max_frequencies=500) for phase in phases])
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

In [ ]:
%%opts Path [aspect='square']
bands = kwant.physics.Bands(syst.leads[1], args=[p])
ks = np.linspace(-3, 3, 300)
Es = np.array([bands(k=k) for k in ks])
hv.Path((ks, Es))[:, -1:1]

In [ ]:
import scipy.sparse.linalg as sla
from scipy.sparse import identity
import kwant.linalg.mumps as mumps

def sparse_diag(matrix, k, sigma, **kwargs):
    """Call sla.eigsh with mumps support.
    Please see scipy.sparse.linalg.eigsh for documentation.
    """
    class LuInv(sla.LinearOperator):
        def __init__(self, A):
            inst = mumps.MUMPSContext()
            inst.analyze(A, ordering='pord')
            inst.factor(A)
            self.solve = inst.solve
            sla.LinearOperator.__init__(self, A.dtype, A.shape)
        def _matvec(self, x):
            return self.solve(x.astype(self.dtype))
    opinv = LuInv(matrix - sigma * identity(matrix.shape[0]))
    return sla.eigsh(matrix, k, sigma=sigma, OPinv=opinv, **kwargs)

def spectrum(syst, p):
    ham = syst.hamiltonian_submatrix(args=[p], sparse=True)
    eigenvals, eigenvecs = sparse_diag(ham, 20, 0)
    return np.sort(eigenvals)

In [ ]:
syst, hopping = funcs.make_1d_wire(a=10, L=64, L_sc=40)
p = funcs.make_params(Delta=0.25, alpha=20, mu_sc=1, mu_sm=1)
alphas = np.linspace(0, 50, 100)
p.B_x = 0.5
Es = np.array([spectrum(syst, p) for p.alpha in alphas])

In [ ]:
%%output size=200
%%opts Path [aspect='square'] HLine (color='k')
hv.Path((alphas, np.sort(Es)),  kdims=[r'$\alpha$', 'E'])[:, -p.Delta:p.Delta]